# import libraries & set matplotlib backend

In [1]:
import numpy as np
import networkx as nx
%matplotlib qt 
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.ion()
import numpy as np
import src.conductor as conductor
from importlib import reload

# Test 1D network

In [2]:
reload(conductor)
ntwrk = conductor.Network(staticnodes=[0,9])
noOfNodes = 10
for i in range(noOfNodes-1):
    ntwrk.add_edge_resistor(i,i+1,1)
ntwrk.draw(tViz=2)
for i in range(0,noOfNodes-1):
    ntwrk.star_mesh(i)
ntwrk.draw(tViz=0.1)

# Initialize 2D network

In [3]:
reload(conductor)
def init_sample_network(
    Xlen = 10,
    Ylen = 10,
    rad=0.3):
    start_node = (Xlen//2,0)
    end_node = (Xlen//2,Ylen-1)
    ntwrk = conductor.Network(staticnodes=[start_node,end_node],rad=rad)
    for i in range(Xlen):
        for j in range(Ylen):
            if j+1<Ylen:
                ntwrk.add_edge_resistor((i,j),(i,(j+1)),1)
            if i+1<Xlen:
                if j==0 or j==Ylen-1:
                    ntwrk.add_edge_resistor((i,j),((i+1),j),0)
                else:
                    ntwrk.add_edge_resistor((i,j),((i+1),j),1)
    return ntwrk


# Star-Mesh transform (Brute force)

In [4]:
%%time
reload(conductor)
ntwrk = init_sample_network(Xlen=10,Ylen=10)
ntwrk.draw()

Wall time: 932 ms


In [5]:
%%time
for node in list(ntwrk.nodes):
    ntwrk.star_mesh(node)
    print(f"\rRemoved node:{node} ... ",end='',flush=True)
    ntwrk.draw(annotate_resistors=False)
ntwrk.draw()
print("Done!")

Removed node:(9, 9) ... Done!
Wall time: 11.6 s


In [15]:
%%time
reload(conductor)
ntwrk = init_sample_network(Xlen=100,Ylen=100)
# ntwrk.draw()

Wall time: 153 ms


In [16]:
%%time
for node in list(ntwrk.nodes):
    ntwrk.star_mesh(node)
    # print(f"\rRemoved node:{node} ... ",end='',flush=True)
    # ntwrk.draw(annotate_resistors=False)
# ntwrk.draw()
print("Done!")

Done!
Wall time: 3min 22s


# Bond propagation

In [6]:
def propagate(node1,node2,node3, update_plot=True, tViz=0.001):
    newnode = tuple(map(np.mean,zip(node1,node2,node3)))
    node = ntwrk.delta_wye(node1,node2,node3,newnode)
    if update_plot:
        ntwrk.pos[node] = ntwrk.node_to_pos(*node)
        ntwrk.draw(annotate_resistors=False,tViz=tViz)
    ntwrk.star_mesh(node3)
    if update_plot:
        ntwrk.draw(annotate_resistors=False,tViz=tViz)
    ntwrk.shift_node(node,node3)
    if update_plot:
        ntwrk.draw(annotate_resistors=False,tViz=tViz)

In [11]:
%%time
reload(conductor)
ntwrk = init_sample_network(Xlen=10,Ylen=10, rad=0.2)
ntwrk.draw()

Wall time: 598 ms


In [12]:
%%time
Xlen,Ylen = np.max(ntwrk.nodes,axis=0) + 1
update_on = True
for i in range(Xlen):
    ntwrk.star_mesh((i,Ylen-1))
    if update_on:
        ntwrk.draw(annotate_resistors=False,tViz=0.0001)
    ntwrk.star_mesh((i,0))
    if update_on:
        ntwrk.draw(annotate_resistors=False,tViz=0.0001)
for j in range(1,Ylen-1):
    for i in range(Xlen):
        if ntwrk.has_edge((i+1,j),(i,j)) and ntwrk[(i+1,j)][(i,j)]['resistance']==0:
            if not ntwrk.merge_short((i+1,j),(i,j)):
                continue
        else:
            ntwrk.star_mesh((i,j))
        if update_on:
            ntwrk.draw(annotate_resistors=False,tViz=0.0001)
        ii=i
        jj=j
        while ii+1<Xlen and jj+1<Ylen-1:
            if update_on:
                print(f"\rPropagating edge:({ii},{jj+1}),({ii+1},{jj}) ... ", end='', flush=True)
            propagate((ii,jj+1),(ii+1,jj),(ii+1,jj+1), tViz=0.0001, update_plot=update_on)
            ii+=1
            jj+=1
if update_on:
    ntwrk.draw()
print("Done!")

Done!
Wall time: 1min 6s


In [13]:
%%time
reload(conductor)
ntwrk = init_sample_network(Xlen=100,Ylen=100, rad=0.2)
# ntwrk.draw()

Wall time: 134 ms


In [14]:
%%time
Xlen,Ylen = np.max(ntwrk.nodes,axis=0) + 1
update_on = False
for i in range(Xlen):
    ntwrk.star_mesh((i,Ylen-1))
    if update_on:
        ntwrk.draw(annotate_resistors=False,tViz=0.0001)
    ntwrk.star_mesh((i,0))
    if update_on:
        ntwrk.draw(annotate_resistors=False,tViz=0.0001)
for j in range(1,Ylen-1):
    for i in range(Xlen):
        if ntwrk.has_edge((i+1,j),(i,j)) and ntwrk[(i+1,j)][(i,j)]['resistance']==0:
            if not ntwrk.merge_short((i+1,j),(i,j)):
                continue
        else:
            ntwrk.star_mesh((i,j))
        if update_on:
            ntwrk.draw(annotate_resistors=False,tViz=0.0001)
        ii=i
        jj=j
        while ii+1<Xlen and jj+1<Ylen-1:
            if update_on:
                print(f"\rPropagating edge:({ii},{jj+1}),({ii+1},{jj}) ... ", end='', flush=True)
            propagate((ii,jj+1),(ii+1,jj),(ii+1,jj+1), tViz=0.0001, update_plot=update_on)
            ii+=1
            jj+=1
if update_on:
    ntwrk.draw()
print("Done!")

Done!
Wall time: 42.9 s


In [115]:
reload(conductor)
def Risotropic(S, dir):
    return (np.cos(np.pi*S)+3)/2
def Rnematic(S, dir):
    return ((np.cos(np.pi*S)+3)/2)*dir + ((-np.cos(np.pi*S)+3)/2)*(1-dir)
# latticeData = np.random.rand(4,4)
latticeData = np.ones((10,10))

In [131]:
reload(conductor)
latticeData = np.ones((40,20))
for terminal_start in ['left','right','up','down']:
    for terminal_end in ['left','right','up','down']:
        # if terminal_end=='down':
            grid2d = conductor.Lattice2DGrid(latticeData=latticeData,
                Rfunc=Risotropic,
                terminal_start=terminal_start, # 'left','right','up','down'
                terminal_end=terminal_end, # 'left','right','up','down'
                rad=0)
            grid2d.BondPropagation2D(update_plot=False,annotate_resistors=False,annotate_nodes=True,tViz=0.0001)
            # break
    # break

left<->left: Shorted measurement terminals!
left<->left: R = 0
left<->right: R = 4.000000000000003
left<->right: Time Elapsed = 1.2 seconds
left<->up: R = 0.648195938699233
left<->up: Time Elapsed = 1.8 seconds
left<->down: R = 0.648195938699233
left<->down: Time Elapsed = 1.8 seconds
right<->left: R = 4.000000000000003
right<->left: Time Elapsed = 1.0 seconds
right<->right: Shorted measurement terminals!
right<->right: R = 0
right<->up: R = 0.648195938699233
right<->up: Time Elapsed = 1.8 seconds
right<->down: R = 0.648195938699233
right<->down: Time Elapsed = 1.8 seconds
up<->left: R = 0.648195938699233
up<->left: Time Elapsed = 2.0 seconds
up<->right: R = 0.648195938699233
up<->right: Time Elapsed = 2.0 seconds
up<->up: Shorted measurement terminals!
up<->up: R = 0
up<->down: R = 1.0
up<->down: Time Elapsed = 1.5 seconds
down<->left: R = 0.648195938699233
down<->left: Time Elapsed = 1.8 seconds
down<->right: R = 0.648195938699233
down<->right: Time Elapsed = 1.7 seconds
down<->up: R

In [112]:
reload(conductor)
for terminal_start in ['left','right','up','down']:
    for terminal_end in ['left','right','up','down']:
        grid2d = conductor.Lattice2DGrid(latticeData=latticeData,
            Rfunc=Rnematic,
            terminal_start=terminal_start, # 'left','right','up','down'
            terminal_end=terminal_end, # 'left','right','up','down'
            rad=0)
        grid2d.BondPropagation2D(update_plot=False,annotate_nodes=False,tViz=0.0001)
    #     if terminal_end=='down':
    #         break
    # break

left<->left: Shorted measurement terminals!
left<->left: R = 0
left<->right: R = 3.999999999999999
left<->up: R = 1.48721740781847
left<->down: R = 1.48721740781847
right<->left: R = 3.999999999999999
right<->right: Shorted measurement terminals!
right<->right: R = 0
right<->up: R = 1.48721740781847
right<->down: R = 1.48721740781847
up<->left: R = 1.48721740781847
up<->right: R = 1.48721740781847
up<->up: Shorted measurement terminals!
up<->up: R = 0
up<->down: R = 2.0
down<->left: R = 1.48721740781847
down<->right: R = 1.48721740781847
down<->up: R = 2.0
down<->down: Shorted measurement terminals!
down<->down: R = 0


---
# END